# **FraudCluster - Fraudulent Bank Transaction Detection Model with Exploratory Data Analysis - Using K-Means Clustering**

---

**Alam Rincon - [GitHub: MrRincon](https://github.com/MrRincon)**

**Petar Atanasov - [GitHub: petar-Atanasov](https://github.com/petar-Atanasov)**

**Teon Morgan - [GitHub: Mi1kDev](https://github.com/Mi1kDev)**

---

**Khorasani, V. (2024) 'Bank Transaction Dataset for Fraud Detection'. Available at: https://www.kaggle.com/datasets/valakhorasani/bank-transaction-dataset-for-fraud-detection (Accessed: 24 June 2025). Available license at: https://www.apache.org/licenses/LICENSE-2.0.**

## Problem Definition

## Data Preprocessing

### Feature Selection and Augmentation

## Exploratory Data Analysis

## Model Development and Evaluation

### K-Means for Fraud Detection

### Outlier Identification

## Analysing Clusters

## Ethical Consideration